<a href="https://colab.research.google.com/github/WillyKitheka/code-doyen/blob/main/understat_timeSeries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.statespace.varmax import VARMAX
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import grangercausalitytests, adfuller
from tqdm import tqdm_notebook
from itertools import product

import matplotlib.pyplot as plt
import statsmodels.api as sm
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# bring in the data to male the soup!!!!
# the parse_date = True argument is used here to avoid the date column from being read as an object!

import pandas as pd

complete_understat = pd.read_csv('/content/drive/MyDrive/xgboost_data/data_understat/complete_understat_per_game.csv', parse_dates = True)

In [ ]:
complete_understat

,league,year,h_a,xG,xGA,npxG,npxGA,deep,deep_allowed,scored,...,ppda_coef,ppda_att,ppda_def,oppda_coef,oppda_att,oppda_def,team,xG_diff,xGA_diff,xpts_diff
0,La_liga,2014,h,1.321070,1.141510,0.438073,1.141510,4,5,1,...,12.071429,338,28,6.300000,189,30,Malaga,0.321070,1.141510,-1.4697
1,La_liga,2014,a,1.131960,1.968630,1.131960,1.069640,4,5,0,...,15.368421,292,19,6.066667,182,30,Malaga,1.131960,-1.031370,0.7086
2,La_liga,2014,h,0.819725,0.358265,0.819725,0.358265,7,1,0,...,9.846154,128,13,8.925926,241,27,Malaga,0.819725,0.358265,0.7526
3,La_liga,2014,a,0.446347,2.761880,0.446347,2.761880,6,2,2,...,5.500000,99,18,8.000000,264,33,Malaga,-1.553653,0.761880,-0.9372
4,La_liga,2014,h,0.278657,0.646364,0.278657,0.646364,4,7,0,...,9.451613,293,31,4.571429,96,21,Malaga,0.278657,0.646364,-0.1279
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40197,RFPL,2023,a,1.085150,0.659563,1.085150,0.659563,10,5,0,...,5.482759,159,29,10.294118,175,17,Fakel,1.085150,-1.340437,1.7434
40198,RFPL,2023,h,1.089200,0.499743,1.089200,0.499743,6,4,1,...,5.318182,117,22,5.384615,140,26,Fakel,0.089200,-0.500257,0.8783
40199,RFPL,2023,h,1.285580,2.388220,0.531300,0.730445,7,4,1,...,5.400000,135,25,7.933333,119,15,Fakel,0.285580,-0.611780,0.4854
40200,RFPL,2023,a,1.133150,1.465410,0.378876,1.465410,1,12,2,...,36.125000,289,8,6.500000,117,18,Fakel,-0.866850,1.465410,-1.9013


In [ ]:
complete_understat.columns

Index(['league', 'year', 'h_a', 'xG', 'xGA', 'npxG', 'npxGA', 'deep',
       'deep_allowed', 'scored', 'missed', 'xpts', 'result', 'date', 'wins',
       'draws', 'loses', 'pts', 'npxGD', 'ppda_coef', 'ppda_att', 'ppda_def',
       'oppda_coef', 'oppda_att', 'oppda_def', 'team', 'xG_diff', 'xGA_diff',
       'xpts_diff'],
      dtype='object')

In [ ]:
complete_understat.dtypes

league           object
year              int64
h_a              object
xG              float64
xGA             float64
npxG            float64
npxGA           float64
deep              int64
deep_allowed      int64
scored            int64
missed            int64
xpts            float64
result           object
date             object
wins              int64
draws             int64
loses             int64
pts               int64
npxGD           float64
ppda_coef       float64
ppda_att          int64
ppda_def          int64
oppda_coef      float64
oppda_att         int64
oppda_def         int64
team             object
xG_diff         float64
xGA_diff        float64
xpts_diff       float64
dtype: object

In [ ]:
data_understat = complete_understat.drop(['scored', 'missed', 'wins', 'draws','loses','pts'], axis=1)

In [ ]:
# Print the distinct values of the 'result' column
print(data_understat['team'].unique())

['Malaga' 'Sevilla' 'Deportivo La Coruna' 'Real Sociedad' 'Espanyol'
 'Getafe' 'Atletico Madrid' 'Rayo Vallecano' 'Valencia' 'Athletic Club'
 'Barcelona' 'Real Madrid' 'Levante' 'Celta Vigo' 'Villarreal' 'Granada'
 'Eibar' 'Cordoba' 'Elche' 'Almeria' 'Las Palmas' 'Sporting Gijon'
 'Real Betis' 'Osasuna' 'Alaves' 'Leganes' 'Girona' 'Real Valladolid'
 'SD Huesca' 'Mallorca' 'Cadiz' 'Aston Villa' 'Everton' 'Southampton'
 'Leicester' 'West Bromwich Albion' 'Sunderland' 'Crystal Palace'
 'Chelsea' 'West Ham' 'Tottenham' 'Arsenal' 'Swansea' 'Stoke'
 'Newcastle United' 'Liverpool' 'Manchester City' 'Manchester United'
 'Hull' 'Burnley' 'Queens Park Rangers' 'Bournemouth' 'Norwich' 'Watford'
 'Middlesbrough' 'Huddersfield' 'Brighton' 'Cardiff' 'Fulham'
 'Wolverhampton Wanderers' 'Sheffield United' 'Leeds' 'Brentford'
 'Nottingham Forest' 'Bayern Munich' 'Hamburger SV' 'Bayer Leverkusen'
 'Hoffenheim' 'Augsburg' 'Hertha Berlin' 'Werder Bremen' 'Schalke 04'
 'Mainz 05' 'Hannover 96' 'Borussia Do

In [ ]:
!pip install pandas-profiling


In [ ]:
# we have found out above, what the shape of the df is, what the names of the columns are
# and what the dtypes of the various columns are...now we can run pandas profiling 
# to have a more intimate knowledge of the data, under the 'know_thy_data' principle...in come chatGPT!

import pandas as pd
from pandas_profiling import ProfileReport

# Load the dataset into a Pandas dataframe
#df = pd.read_csv("path/to/your/dataset.csv")

# Generate the report
understat_profile_report = ProfileReport(complete_understat, title="Understat_data_Pandas_Profiling_Report", explorative=True)

# Save the report as an HTML file
understat_profile_report.to_file("/content/drive/MyDrive/xgboost_data/data_understat/understat_profile_report.html")



<ipython-input-5-5938324a20f2>:6: DeprecationWarning: `import pandas_profiling` is going to be deprecated by April 1st. Please use `import ydata_profiling` instead.
  from pandas_profiling import ProfileReport


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
understat_profile_report

In [ ]:
data_understat

,league,year,h_a,xG,xGA,npxG,npxGA,deep,deep_allowed,xpts,...,ppda_coef,ppda_att,ppda_def,oppda_coef,oppda_att,oppda_def,team,xG_diff,xGA_diff,xpts_diff
0,La_liga,2014,h,1.321070,1.141510,0.438073,1.141510,4,5,1.5303,...,12.071429,338,28,6.300000,189,30,Malaga,0.321070,1.141510,-1.4697
1,La_liga,2014,a,1.131960,1.968630,1.131960,1.069640,4,5,0.7086,...,15.368421,292,19,6.066667,182,30,Malaga,1.131960,-1.031370,0.7086
2,La_liga,2014,h,0.819725,0.358265,0.819725,0.358265,7,1,1.7526,...,9.846154,128,13,8.925926,241,27,Malaga,0.819725,0.358265,0.7526
3,La_liga,2014,a,0.446347,2.761880,0.446347,2.761880,6,2,0.0628,...,5.500000,99,18,8.000000,264,33,Malaga,-1.553653,0.761880,-0.9372
4,La_liga,2014,h,0.278657,0.646364,0.278657,0.646364,4,7,0.8721,...,9.451613,293,31,4.571429,96,21,Malaga,0.278657,0.646364,-0.1279
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40197,RFPL,2023,a,1.085150,0.659563,1.085150,0.659563,10,5,1.7434,...,5.482759,159,29,10.294118,175,17,Fakel,1.085150,-1.340437,1.7434
40198,RFPL,2023,h,1.089200,0.499743,1.089200,0.499743,6,4,1.8783,...,5.318182,117,22,5.384615,140,26,Fakel,0.089200,-0.500257,0.8783
40199,RFPL,2023,h,1.285580,2.388220,0.531300,0.730445,7,4,0.4854,...,5.400000,135,25,7.933333,119,15,Fakel,0.285580,-0.611780,0.4854
40200,RFPL,2023,a,1.133150,1.465410,0.378876,1.465410,1,12,1.0987,...,36.125000,289,8,6.500000,117,18,Fakel,-0.866850,1.465410,-1.9013


In [ ]:
data_understat['date'] = pd.to_datetime(data_understat['date'], format='%Y-%m-%d %H:%M:%S')


In [ ]:
data_understat

,league,year,h_a,xG,xGA,npxG,npxGA,deep,deep_allowed,xpts,...,ppda_coef,ppda_att,ppda_def,oppda_coef,oppda_att,oppda_def,team,xG_diff,xGA_diff,xpts_diff
0,La_liga,2014,h,1.321070,1.141510,0.438073,1.141510,4,5,1.5303,...,12.071429,338,28,6.300000,189,30,Malaga,0.321070,1.141510,-1.4697
1,La_liga,2014,a,1.131960,1.968630,1.131960,1.069640,4,5,0.7086,...,15.368421,292,19,6.066667,182,30,Malaga,1.131960,-1.031370,0.7086
2,La_liga,2014,h,0.819725,0.358265,0.819725,0.358265,7,1,1.7526,...,9.846154,128,13,8.925926,241,27,Malaga,0.819725,0.358265,0.7526
3,La_liga,2014,a,0.446347,2.761880,0.446347,2.761880,6,2,0.0628,...,5.500000,99,18,8.000000,264,33,Malaga,-1.553653,0.761880,-0.9372
4,La_liga,2014,h,0.278657,0.646364,0.278657,0.646364,4,7,0.8721,...,9.451613,293,31,4.571429,96,21,Malaga,0.278657,0.646364,-0.1279
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40197,RFPL,2023,a,1.085150,0.659563,1.085150,0.659563,10,5,1.7434,...,5.482759,159,29,10.294118,175,17,Fakel,1.085150,-1.340437,1.7434
40198,RFPL,2023,h,1.089200,0.499743,1.089200,0.499743,6,4,1.8783,...,5.318182,117,22,5.384615,140,26,Fakel,0.089200,-0.500257,0.8783
40199,RFPL,2023,h,1.285580,2.388220,0.531300,0.730445,7,4,0.4854,...,5.400000,135,25,7.933333,119,15,Fakel,0.285580,-0.611780,0.4854
40200,RFPL,2023,a,1.133150,1.465410,0.378876,1.465410,1,12,1.0987,...,36.125000,289,8,6.500000,117,18,Fakel,-0.866850,1.465410,-1.9013


In [ ]:
data_understat.dtypes

league                  object
year                     int64
h_a                     object
xG                     float64
xGA                    float64
npxG                   float64
npxGA                  float64
deep                     int64
deep_allowed             int64
xpts                   float64
result                  object
date            datetime64[ns]
npxGD                  float64
ppda_coef              float64
ppda_att                 int64
ppda_def                 int64
oppda_coef             float64
oppda_att                int64
oppda_def                int64
team                    object
xG_diff                float64
xGA_diff               float64
xpts_diff              float64
dtype: object

In [ ]:
data_understat.set_index('date', inplace=True)

In [ ]:
data_understat.index

DatetimeIndex(['2014-08-23 18:00:00', '2014-08-29 21:00:00',
               '2014-09-13 17:00:00', '2014-09-20 21:00:00',
               '2014-09-24 21:00:00', '2014-09-28 11:00:00',
               '2014-10-04 21:00:00', '2014-10-18 21:00:00',
               '2014-10-26 11:00:00', '2014-11-01 21:00:00',
               ...
               '2022-11-12 09:00:00', '2023-03-03 16:00:00',
               '2023-03-11 13:30:00', '2023-03-18 13:30:00',
               '2023-04-02 13:30:00', '2023-04-08 11:00:00',
               '2023-04-16 13:30:00', '2023-04-22 13:30:00',
               '2023-04-28 17:00:00', '2023-05-06 16:00:00'],
              dtype='datetime64[ns]', name='date', length=40202, freq=None)

In [ ]:
var_df = data_understat[['xG', 'xGA', 'npxG', 'npxGA', 'deep', 'deep_allowed', 'xpts', 'npxGD', 'team', 'ppda_coef', 'ppda_att', 'ppda_def', 'oppda_coef', 'oppda_att', 'oppda_def', 'xG_diff', 'xGA_diff', 'xpts_diff']]


In [ ]:
var_df

,xG,xGA,npxG,npxGA,deep,deep_allowed,xpts,npxGD,team,ppda_coef,ppda_att,ppda_def,oppda_coef,oppda_att,oppda_def,xG_diff,xGA_diff,xpts_diff
date,,,,,,,,,,,,,,,,,,
2014-08-23 18:00:00,1.321070,1.141510,0.438073,1.141510,4,5,1.5303,-0.703437,Malaga,12.071429,338,28,6.300000,189,30,0.321070,1.141510,-1.4697
2014-08-29 21:00:00,1.131960,1.968630,1.131960,1.069640,4,5,0.7086,0.062320,Malaga,15.368421,292,19,6.066667,182,30,1.131960,-1.031370,0.7086
2014-09-13 17:00:00,0.819725,0.358265,0.819725,0.358265,7,1,1.7526,0.461460,Malaga,9.846154,128,13,8.925926,241,27,0.819725,0.358265,0.7526
2014-09-20 21:00:00,0.446347,2.761880,0.446347,2.761880,6,2,0.0628,-2.315533,Malaga,5.500000,99,18,8.000000,264,33,-1.553653,0.761880,-0.9372
2014-09-24 21:00:00,0.278657,0.646364,0.278657,0.646364,4,7,0.8721,-0.367707,Malaga,9.451613,293,31,4.571429,96,21,0.278657,0.646364,-0.1279
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-04-08 11:00:00,1.085150,0.659563,1.085150,0.659563,10,5,1.7434,0.425587,Fakel,5.482759,159,29,10.294118,175,17,1.085150,-1.340437,1.7434
2023-04-16 13:30:00,1.089200,0.499743,1.089200,0.499743,6,4,1.8783,0.589457,Fakel,5.318182,117,22,5.384615,140,26,0.089200,-0.500257,0.8783
2023-04-22 13:30:00,1.285580,2.388220,0.531300,0.730445,7,4,0.4854,-0.199145,Fakel,5.400000,135,25,7.933333,119,15,0.285580,-0.611780,0.4854


In [ ]:
var_df.columns

Index(['xG', 'xGA', 'npxG', 'npxGA', 'deep', 'deep_allowed', 'xpts', 'npxGD',
       'team', 'ppda_coef', 'ppda_att', 'ppda_def', 'oppda_coef', 'oppda_att',
       'oppda_def', 'xG_diff', 'xGA_diff', 'xpts_diff'],
      dtype='object')

In [ ]:
# then we extract data for the matches of interest from the data set...

#import pandas as pd

# Load your dataset into a Pandas DataFrame (assuming it's in a CSV file)
#data = pd.read_csv('your_dataset.csv')

# Extract all rows where the "team" column contains the string "Arsenal" etc
arsenal_data = var_df[(var_df['team'] == 'Arsenal')]

# Print the extracted data to verify that it's correct
print(arsenal_data)


                           xG       xGA      npxG     npxGA  deep  \
date                                                                
2014-08-16 17:30:00  1.554110  0.158151  1.554110  0.158151     9   
2014-08-23 17:30:00  1.433550  1.334290  1.433550  1.334290     8   
2014-08-31 16:00:00  2.296490  0.809743  2.296490  0.809743    13   
2014-09-13 12:45:00  2.078040  2.308340  2.078040  2.308340     9   
2014-09-20 15:00:00  1.362240  0.649013  1.362240  0.649013     7   
...                       ...       ...       ...       ...   ...   
2023-04-16 13:00:00  2.717820  1.960950  1.956650  1.199780    10   
2023-04-21 19:00:00  2.396090  1.430630  2.396090  1.430630    23   
2023-04-26 19:00:00  0.398372  2.011680  0.398372  2.011680     6   
2023-05-02 19:00:00  2.783200  0.785202  2.783200  0.785202    13   
2023-05-07 15:30:00  1.290610  1.372240  1.290610  1.372240     8   

                     deep_allowed    xpts     npxGD     team  ppda_coef  \
date                       

In [ ]:
arsenal_data = arsenal_data.drop(['team'], axis=1)

In [ ]:
print(arsenal_data.shape)

(374, 17)


In [ ]:
arsenal_data.columns

Index(['xG', 'xGA', 'npxG', 'npxGA', 'deep', 'deep_allowed', 'xpts', 'npxGD',
       'ppda_coef', 'ppda_att', 'ppda_def', 'oppda_coef', 'oppda_att',
       'oppda_def', 'xG_diff', 'xGA_diff', 'xpts_diff'],
      dtype='object')

In [ ]:
arsenal_data.dtypes

xG              float64
xGA             float64
npxG            float64
npxGA           float64
deep              int64
deep_allowed      int64
xpts            float64
npxGD           float64
ppda_coef       float64
ppda_att          int64
ppda_def          int64
oppda_coef      float64
oppda_att         int64
oppda_def         int64
xG_diff         float64
xGA_diff        float64
xpts_diff       float64
dtype: object

In [ ]:
#performing stationarity test the chatGPT way

from statsmodels.tsa.vector_ar.var_model import VAR
from statsmodels.tsa.stattools import adfuller

# Check for stationarity
def adfuller_test(series, signif=0.05):
    """
    Perform ADFuller to test for stationarity
    """
    adf_test = adfuller(series, autolag='AIC')
    p_value = adf_test[1]
    if p_value < signif:
        print(f"ADF Test: Reject the null hypothesis, data is stationary (p={p_value})")
    else:
        print(f"ADF Test: Fail to reject the null hypothesis, data is non-stationary (p={p_value})")



In [ ]:
from statsmodels.tsa.stattools import adfuller

for col in arsenal_data.columns:
    result = adfuller(arsenal_data[col])
    print(f'Column: {col}')
    print(f'ADF Statistic: {result[0]}')
    print(f'p-value: {result[1]}')
    print(f'Critical Values:')
    for key, value in result[4].items():
        print(f'   {key}: {value}')


In [ ]:
# Split data into training and testing sets
train_size = int(len(arsenal_data) * 0.9)
train_data, test_data = arsenal_data.iloc[:train_size], arsenal_data.iloc[train_size:]

# Train VAR model
model = VAR(train_data)
model_fit = model.fit()

# Make predictions on test set
lag_order = model_fit.k_ar
input_data = test_data.values
predictions = model_fit.forecast(input_data, len(test_data))

# Calculate RMSE for each variable
mse = ((predictions - test_data) ** 2).mean(axis=0)
rmse = np.sqrt(mse)

print('RMSE for each variable:')
for i, col in enumerate(arsenal_data.columns):
    print(col + ':', round(rmse[i], 2))


RMSE for each variable:
xG: 0.89
xGA: 0.86
npxG: 0.87
npxGA: 0.78
deep: 6.15
deep_allowed: 3.9
xpts: 0.88
npxGD: 1.38
ppda_coef: 3.43
ppda_att: 77.22
ppda_def: 4.24
oppda_coef: 4.69
oppda_att: 64.43
oppda_def: 5.75
xG_diff: 0.85
xGA_diff: 1.15
xpts_diff: 0.87


In [ ]:
import pandas as pd

arsenal_data_rmse = pd.DataFrame({'Variable': arsenal_data.columns, 'RMSE': rmse})
arsenal_data_rmse = arsenal_data_rmse.set_index('Variable')
arsenal_data_rmse


,RMSE
Variable,
xG,0.887894
xGA,0.863261
npxG,0.867181
npxGA,0.777992
deep,6.154875
deep_allowed,3.896049
xpts,0.884422
npxGD,1.378176
ppda_coef,3.431064


In [ ]:
arsenal_data_rmse.to_csv('/content/drive/MyDrive/xgboost_data/data_understat/Multivariate-Time-Series-Forecasting-main/arsenal_data_rmse.csv')

In [ ]:
arsenal_data

,xG,xGA,npxG,npxGA,deep,deep_allowed,xpts,npxGD,ppda_coef,ppda_att,ppda_def,oppda_coef,oppda_att,oppda_def,xG_diff,xGA_diff,xpts_diff
date,,,,,,,,,,,,,,,,,
2014-08-16 17:30:00,1.554110,0.158151,1.554110,0.158151,9,0,2.7946,1.395959,2.933333,88,30,14.172414,411,29,-0.445890,-0.841849,-0.2054
2014-08-23 17:30:00,1.433550,1.334290,1.433550,1.334290,8,3,1.4293,0.099260,7.787879,257,33,15.882353,270,17,-0.566450,-0.665710,0.4293
2014-08-31 16:00:00,2.296490,0.809743,2.296490,0.809743,13,4,2.4128,1.486747,7.500000,150,20,11.000000,341,31,1.296490,-0.190257,1.4128
2014-09-13 12:45:00,2.078040,2.308340,2.078040,2.308340,9,16,1.2271,-0.230300,7.081081,262,37,5.022727,221,44,0.078040,0.308340,0.2271
2014-09-20 15:00:00,1.362240,0.649013,1.362240,0.649013,7,0,2.0028,0.713227,7.727273,170,22,44.250000,531,12,-1.637760,0.649013,-0.9972
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-04-16 13:00:00,2.717820,1.960950,1.956650,1.199780,10,4,1.9780,0.756870,6.500000,104,16,15.800000,316,20,0.717820,-0.039050,0.9780
2023-04-21 19:00:00,2.396090,1.430630,2.396090,1.430630,23,3,2.0364,0.965460,4.160000,104,25,18.928571,265,14,-0.603910,-1.569370,1.0364
2023-04-26 19:00:00,0.398372,2.011680,0.398372,2.011680,6,9,0.2988,-1.613308,16.842105,320,19,10.464286,293,28,-0.601628,-1.988320,0.2988


In [ ]:
# Extract all rows where the "team" column contains the string "New Castle" etc
new_castle_data = var_df[(var_df['team'] == 'Newcastle United')]

new_castle_data = new_castle_data.drop(['team'], axis=1)

# Extract all rows where the "team" column contains the string "Milan" etc
milan_data = var_df[(var_df['team'] == 'AC Milan')]

milan_data = milan_data.drop(['team'], axis=1)

# Extract all rows where the "team" column contains the string "Lazio" etc
lazio_data = var_df[(var_df['team'] == 'Lazio')]

lazio_data = lazio_data.drop(['team'], axis=1)

# Extract all rows where the "team" column contains the string "Hoffenheim" etc
hoffenheim_data = var_df[(var_df['team'] == 'Hoffenheim')]


hoffenheim_data = hoffenheim_data.drop(['team'], axis=1)

# Extract all rows where the "team" column contains the string "Eitracht F." etc
eintracht_f_data = var_df[(var_df['team'] == 'Eintracht Frankfurt')]

eintracht_f_data = eintracht_f_data.drop(['team'], axis=1)

# Extract all rows where the "team" column contains the string "Augsburg" etc
augsburg_data = var_df[(var_df['team'] == 'Augsburg')]

augsburg_data = augsburg_data.drop(['team'], axis=1)

# Extract all rows where the "team" column contains the string "Union Berlin" etc
union_berlin_data = var_df[(var_df['team'] == 'Union Berlin')]

union_berlin_data = union_berlin_data.drop(['team'], axis=1)

# Extract all rows where the "team" column contains the string "Hertha" etc
hertha_data = var_df[(var_df['team'] == 'Hertha Berlin')]

hertha_data = hertha_data.drop(['team'], axis=1)

# Extract all rows where the "team" column contains the string "Stutgart" etc
stuttgart_data = var_df[(var_df['team'] == 'VfB Stuttgart')]

stuttgart_data = stuttgart_data.drop(['team'], axis=1)

# Extract all rows where the "team" column contains the string "Wolves" etc
wolves_data = var_df[(var_df['team'] == 'Wolverhampton Wanderers')]

wolves_data = wolves_data.drop(['team'], axis=1)

# Extract all rows where the "team" column contains the string "Aston Villa" etc
aston_villa_data = var_df[(var_df['team'] == 'Aston Villa')]

aston_villa_data = aston_villa_data.drop(['team'], axis=1)

# Extract all rows where the "team" column contains the string "Nice" etc
nice_data = var_df[(var_df['team'] == 'Nice')]

nice_data = nice_data.drop(['team'], axis=1)

# Extract all rows where the "team" column contains the string "Rennes" etc
rennes_data = var_df[(var_df['team'] == 'Rennes')]

rennes_data = rennes_data.drop(['team'], axis=1)

# Extract all rows where the "team" column contains the string "Roma" etc
roma_data = var_df[(var_df['team'] == 'Roma')]

roma_data = roma_data.drop(['team'], axis=1)

# Extract all rows where the "team" column contains the string "Inter" etc
inter_data = var_df[(var_df['team'] == 'Inter')]

inter_data = inter_data.drop(['team'], axis=1)

# Extract all rows where the "team" column contains the string "Reims" etc
reims_data = var_df[(var_df['team'] == 'Reims')]

reims_data = reims_data.drop(['team'], axis=1)

# Extract all rows where the "team" column contains the string "Lille" etc
lille_data = var_df[(var_df['team'] == 'Lille')]

lille_data = lille_data.drop(['team'], axis=1)

# Extract all rows where the "team" column contains the string "Cremonese" etc
cremonese_data = var_df[(var_df['team'] == 'Cremonese')]

cremonese_data = cremonese_data.drop(['team'], axis=1)

# Extract all rows where the "team" column contains the string "Spezia" etc
spezia_data = var_df[(var_df['team'] == 'Spezia')]

spezia_data = spezia_data.drop(['team'], axis=1)

# Extract all rows where the "team" column contains the string "Lens" etc
lens_data = var_df[(var_df['team'] == 'Lens')]

lens_data = lens_data.drop(['team'], axis=1)

# Extract all rows where the "team" column contains the string "Marseille" etc
marseille_data = var_df[(var_df['team'] == 'Marseille')]

marseille_data = marseille_data.drop(['team'], axis=1)

# Extract all rows where the "team" column contains the string "Atalanta" etc
atalanta_data = var_df[(var_df['team'] == 'Atalanta')]

atalanta_data = atalanta_data.drop(['team'], axis=1)

# Extract all rows where the "team" column contains the string "Juventus" etc
juventus_data = var_df[(var_df['team'] == 'Juventus')]

juventus_data = juventus_data.drop(['team'], axis=1)

# Extract all rows where the "team" column contains the string "Nantes" etc
nantes_data = var_df[(var_df['team'] == 'Nantes')]

nantes_data = nantes_data.drop(['team'], axis=1)

# Extract all rows where the "team" column contains the string "Strasbourg" etc
strasbourg_data = var_df[(var_df['team'] == 'Strasbourg')]

strasbourg_data = strasbourg_data.drop(['team'], axis=1)

# Extract all rows where the "team" column contains the string "West Ham" etc
west_ham_data = var_df[(var_df['team'] == 'West Ham')]

west_ham_data = west_ham_data.drop(['team'], axis=1)

# Extract all rows where the "team" column contains the string "Man Utd" etc
man_utd_data = var_df[(var_df['team'] == 'Manchester United')]

man_utd_data = man_utd_data.drop(['team'], axis=1)

# Extract all rows where the "team" column contains the string "Lecce" etc
lecce_data = var_df[(var_df['team'] == 'Lecce')]

lecce_data = lecce_data.drop(['team'], axis=1)

# Extract all rows where the "team" column contains the string "Hellas Verona" etc
hellas_verona_data = var_df[(var_df['team'] == 'Verona')]

hellas_verona_data = hellas_verona_data.drop(['team'], axis=1)

In [ ]:
# creating the vars_df_dict

vars_df_dict = {'arsenal': arsenal_data, 'new_castle_data': new_castle_data,'milan_data': milan_data, 'lazio_data': lazio_data,
                'hoffenheim_data': hoffenheim_data, 'eintracht_f_data':eintracht_f_data, 'augsburg_data':augsburg_data, 'union_berlin_data':union_berlin_data,
                'hertha_data':hertha_data, 'stuttgart_data':stuttgart_data, 'wolves_data':wolves_data, 'aston_villa_data':aston_villa_data, 'nice_data':nice_data,
                'rennes_data':rennes_data, 'roma_data':roma_data, 'inter_data':inter_data, 'reims_data':reims_data,'lille_data':lille_data,'cremonese_data':cremonese_data,
                'spezia_data':spezia_data,'lens_data':lens_data, 'marseille_data':marseille_data,'atalanta_data':atalanta_data,'juventus_data':juventus_data,
                'nantes_data':nantes_data,'strasbourg_data':strasbourg_data,'west_ham_data':west_ham_data,'man_utd_data':man_utd_data,'lecce_data':lecce_data,
                'hellas_verona_data':hellas_verona_data}

In [ ]:
def train_test_var(vars_df_dict):
    all_rmse_df = pd.DataFrame()
    
    for key in vars_df_dict.keys():
        df = vars_df_dict[key]
        
        # Check if dataframe is empty
        if df.empty:
            print(f"{key} dataframe is empty. Skipping...")
            continue
        
        # Split data into training and testing sets
        train_size = int(len(df) * 0.9)
        train_data, test_data = df.iloc[:train_size], df.iloc[train_size:]

        # Train VAR model
        model = VAR(train_data)
        model_fit = model.fit()

        # Make predictions on test set
        lag_order = model_fit.k_ar
        input_data = test_data.values
        predictions = model_fit.forecast(input_data, len(test_data))

        # Calculate RMSE for each variable
        mse = ((predictions - test_data) ** 2).mean(axis=0)
        rmse = np.sqrt(mse)

        # Save RMSE for this dataframe
        df_rmse = pd.DataFrame({'Variable': df.columns, 'RMSE': rmse})
        df_rmse = df_rmse.set_index('Variable')
        df_rmse.columns = [key]
        all_rmse_df = pd.concat([all_rmse_df, df_rmse], axis=1)
        
    return all_rmse_df


In [ ]:
all_rmse_df = train_test_var(vars_df_dict)
print(all_rmse_df)


In [ ]:
all_rmse_df

,arsenal,new_castle_data,milan_data,lazio_data,hoffenheim_data,eintracht_f_data,augsburg_data,union_berlin_data,hertha_data,stuttgart_data,...,lens_data,marseille_data,atalanta_data,juventus_data,nantes_data,strasbourg_data,west_ham_data,man_utd_data,lecce_data,hellas_verona_data
Variable,,,,,,,,,,,,,,,,,,,,,
xG,0.887894,1.374423,0.814458,0.922825,0.997033,1.042049,0.675197,0.588603,0.740429,0.764875,...,0.789731,0.908271,1.006130,0.808219,0.722535,0.716879,0.798402,0.941439,0.789338,0.678701
xGA,0.863261,0.803917,0.607894,0.731049,0.829408,0.862508,0.985877,0.997881,0.898543,0.640229,...,0.629793,0.777948,0.625711,0.674670,0.944756,0.852732,0.887241,0.871200,0.633949,0.799505
npxG,0.867181,1.344151,0.852153,0.678157,0.787104,0.861495,0.553867,0.574081,0.692780,0.724117,...,0.853992,0.920226,0.873132,0.715988,0.593796,0.656122,0.713497,0.967948,0.767623,0.638781
npxGA,0.777992,0.806758,0.529589,0.746803,0.824938,0.832279,0.958328,0.964167,0.886033,0.662848,...,0.641338,0.765861,0.596866,0.612855,0.859763,0.581816,0.776744,0.872031,0.531593,0.742892
deep,6.154875,6.704521,3.907279,2.747899,2.966948,4.004676,2.849718,2.865780,2.372711,3.164488,...,3.379212,5.163230,5.543924,3.307812,2.604505,3.241928,2.781163,4.585392,3.593552,2.314386
deep_allowed,3.896049,3.823552,3.414368,3.092967,4.675736,3.925451,4.090684,4.704213,4.023737,3.894840,...,3.604620,4.011507,2.949876,4.681939,5.961262,3.796727,3.998299,4.374222,3.528265,3.553022
xpts,0.884422,1.128955,0.781231,0.876941,0.915063,0.811656,0.665471,0.691222,0.796935,0.732409,...,0.741621,0.765751,0.836464,0.833891,0.786569,0.819119,0.826937,0.805324,0.761220,0.725346
npxGD,1.378176,1.857702,1.117662,1.127007,1.291845,1.292448,1.046712,1.179564,1.278372,1.108246,...,1.085651,1.278833,1.203291,1.156201,1.129921,1.035759,1.237944,1.401964,0.997760,1.087389
ppda_coef,3.431064,8.210364,3.933063,7.341513,3.609980,5.976432,4.163266,10.123423,6.384490,7.771641,...,5.384366,3.309813,7.685115,6.093475,5.321173,7.831896,8.611944,5.430915,5.123489,10.768868


In [ ]:
all_rmse_df.to_csv('/content/drive/MyDrive/xgboost_data/data_understat/Multivariate-Time-Series-Forecasting-main/all_rmse_df.csv')